In [24]:
from collections import Counter,defaultdict
import re

def tokenize(message):
    message=message.lower()
    all_words=re.findall("[a-z0-9']+", message)
    return set(all_words)

def word_count_old(doocuments):
    return Counter(word
                  for document in documents
                  for word in document)

def wc_mapper(document):
    for word in tokenize(document):
        yield (word,1)
        
def wc_reducer(word,counts):
    yield (word,sum(counts))

def word_count(documents):
    collector=defaultdict(list)
    
    for document in documents:
        for word,count in wc_mapper(document):
            collector[word].append(count)
    return [output
           for word,counts in collector.iteritems()
           for output in wc_reducer(word,counts)]

documents=["data science","big data","science fiction"]

print word_count(documents)

[('science', 2), ('fiction', 1), ('data', 2), ('big', 1)]


In [23]:
def map_reduce(inputs,mapper,reducer):
    collector=defaultdict(list)
    
    for input in inputs:
        for key,value in mapper(input):
            collector[key].append(value)
    return [output for key,values in collector.iteritems()
           for output in reducer(key,values)]

word_counts=map_reduce(documents,wc_mapper,wc_reducer)



def reduce_values_using(aggregation_fn,key,values):
    yield (key,aggregation_fn(values))
    
def values_reducer(aggregation_fn):
    return partial(reduce_values_using,aggregation_fn)

sum_reducer=values_reducer(sum)
max_reducer=values_reducer(max)


NameError: global name 'partial' is not defined

In [27]:
import datetime

status_updates={"id":1,
 "username":"joelgrus",
 "text":"Is anyone interested in a data science book?",
 "created_at": datetime.datetime(2013,12,21,11,47),
 "liked_by":["data_guy","data_girl","mike"]
}

def data_science_day_mapper(status_update):
    if "data science" in status_update["text"].lower():
        day_of_week=status_update["created_at"].weekday()
        yield (day_of_week,1)

data_science_days=map_reduce(status_updates,data_science_day_mapper,sum_reducer)



def words_per_user_mapper(status_update):
    user=status_update["username"]
    for word in tokenize(status_update["text"]):
        yield (user,(word,1))

def most_popular_word_reducer(user,words_and_counts):
    word_counts=Counter()
    for word,count in words_and_counts:
        word_counts[word]+=count
    word,count=word_counts.most_common(1)[0]
    
    yield(user,(word,count))

user_words=map_reduce(status_updates,words_per_user_mapper,most_popular_word_reducer)

def like_mapper(status_update):
    user=status_update["username"]
    for liker in status_update["liked_by"]:
        yield(user,liker)

distinct_likers_per_user=map_reduce(status_updates,liker_mapper,count_distinct_reducer)

NameError: name 'sum_reducer' is not defined

In [ ]:
def matrix_multiply_mapper(m,element):
    name,i,j,value=element
    
    if name=="A":
        for k in range(m):
            yield((i,k),(j,value))
    else:
        for k in range(m):
            yield((k,j),(i,value))

def matrix_multiply_reducer(m,key,indexed_values):
    results_by_index=defaultdict(list)
    for index,value in indexed_values:
        results_by_index[index].append(value)
        
    sum_product=sum(results[0]*results[1]
                   for results in results_by_index.values()
                   if len(results)==2)
    if sum_product !=0.0:
        yield(key,sum_product)

